In [1]:
from gurobipy import *
import pandas as pd
import gurobipy as gp

In [204]:
distances = pd.read_excel('distances.xlsx', index_col=0)
distances = distances.drop(["Price", "X Position", "Y Position", "Item"], axis = 1)
distances = distances.set_axis([idx for idx in distances.index], axis = 1)
#distances[57] = distances[0]
display(distances)

distances = {(i, j): distances.at[i, j] for i in distances.index for j in list(distances.set_index(distances.columns).reset_index(drop = True).index)}

c = distances


,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0,15,35,65,90,100,95,75,65,35,...,135,140,155,185,185,145,135,155,165,195
1,15,0,20,50,75,85,110,90,80,50,...,150,155,170,200,200,160,150,170,180,210
2,35,20,0,30,55,65,110,110,100,70,...,170,175,190,180,200,180,170,190,200,190
3,65,50,30,0,25,35,80,100,110,100,...,200,195,180,150,170,210,200,200,190,160
4,90,75,55,25,0,10,55,75,85,115,...,175,170,155,125,145,185,195,175,165,135
5,100,85,65,35,10,0,45,65,75,105,...,165,160,145,115,135,175,185,165,155,125
6,95,110,110,80,55,45,0,20,30,60,...,170,165,150,120,140,180,190,170,160,130
7,75,90,110,100,75,65,20,0,10,40,...,190,185,170,140,160,200,190,190,180,150
8,65,80,100,110,85,75,30,10,0,30,...,180,185,180,150,170,190,180,200,190,160
9,35,50,70,100,115,105,60,40,30,0,...,150,155,170,180,200,160,150,170,180,190


In [252]:
display(c)

{(0, 0): 0,
 (0, 1): 15,
 (0, 2): 35,
 (0, 3): 65,
 (0, 4): 90,
 (0, 5): 100,
 (0, 6): 95,
 (0, 7): 75,
 (0, 8): 65,
 (0, 9): 35,
 (0, 10): 20,
 (0, 11): 45,
 (0, 12): 65,
 (0, 13): 95,
 (0, 14): 100,
 (0, 15): 105,
 (0, 16): 125,
 (0, 17): 120,
 (0, 18): 115,
 (0, 19): 75,
 (0, 20): 45,
 (0, 21): 60,
 (0, 22): 75,
 (0, 23): 135,
 (0, 24): 125,
 (0, 25): 140,
 (0, 26): 65,
 (0, 27): 75,
 (0, 28): 95,
 (0, 29): 115,
 (0, 30): 145,
 (0, 31): 75,
 (0, 32): 80,
 (0, 33): 95,
 (0, 34): 125,
 (0, 35): 135,
 (0, 36): 155,
 (0, 37): 125,
 (0, 38): 105,
 (0, 39): 100,
 (0, 40): 85,
 (0, 41): 95,
 (0, 42): 105,
 (0, 43): 100,
 (0, 44): 165,
 (0, 45): 180,
 (0, 46): 105,
 (0, 47): 135,
 (0, 48): 140,
 (0, 49): 155,
 (0, 50): 185,
 (0, 51): 185,
 (0, 52): 145,
 (0, 53): 135,
 (0, 54): 155,
 (0, 55): 165,
 (0, 56): 195,
 (1, 0): 15,
 (1, 1): 0,
 (1, 2): 20,
 (1, 3): 50,
 (1, 4): 75,
 (1, 5): 85,
 (1, 6): 110,
 (1, 7): 90,
 (1, 8): 80,
 (1, 9): 50,
 (1, 10): 35,
 (1, 11): 60,
 (1, 12): 80,
 (1, 13):

In [43]:
distances.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56],
           dtype='int64')

In [28]:
distances.shape[1]

58

In [273]:
model = gp.Model("Supermarket_Sweep")

distances = pd.read_excel('distances.xlsx', index_col=0)
n=len(distances)-4
times= pd.read_excel('times.xlsx', index_col=0)
prices= pd.read_excel('times.xlsx')
prices=prices["Price"]
x = {}
y = {}
#distances 
print(n)
t = {}
#time
M = 1000 

for a in range(1, 3):
    for i in range(1, n+2):
        for j in range(1, n+2):
            x[a, i, j] = model.addVar(vtype=GRB.BINARY, name=f"x_{a}_{i}_{j}")
            t[a, i, j] = model.addVar(name=f"t_{a}_{i}_{j}")
for a in range(1, 3):
    for j in range(1, n+2):
        y[a, j] = model.addVar(name=f"y_{a}_{i}_{j}")

# Objective Function
model.setObjective(quicksum(prices[j] * x[a, i, j] for i in range(1, n) for j in range(2, n+2) for a in range(1, 3)), GRB.MAXIMIZE)


#constraint adding for part b formulation

model.addConstr(0.1 * y[1, n+1] + quicksum(2 * x[1, i, j] for i in range(1, n+1) for j in range(1, n+2)) <= 22)
model.addConstr(0.1 * y[2, n+1] + quicksum(2 * x[2, i, j] for i in range(1, n+1) for j in range(1, n+2)) <= 22)

    # Constraint 2
#model.addConstr(quicksum(x[a, i, j] for i in range(1, n) for j in range(2, n+2)) <= 10)
model.addConstr(quicksum(x[1, i, j] for i in range(1, n) for j in range(1, n+2)) <= 9)
model.addConstr(quicksum(x[2, i, j] for i in range(1, n) for j in range(1, n+2)) <= 9)
#model.addConstr(quicksum(x[a, i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 10 for a in range(1, 3))
   
    # Constraint 3
model.addConstr(quicksum(x[a, 1, j] + x[a, 2, j] for j in range(2, n+2)) <= 1)
#model.addConstr(quicksum(x[1, i, j] + x[2, i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 1)
   
#sales person constraints 
model.addConstrs(t[a, i, j] <= M * x[a, i, j] for a in range(1, 3) for i in range(1, n) for j in range(2, n+2))
model.addConstrs(y[a, 1] == 0 for a in range(1, 3))

model.addConstrs(y[1, j] <= quicksum(t[1, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))
model.addConstrs(y[2, j] <= quicksum(t[2, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))
model.addConstrs(y[1, j] >= quicksum(t[1, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))
model.addConstrs(y[2, j] >= quicksum(t[2, i, j] for i in range(1, n + 1)) for j in range(2, n + 1) for j in range(1, n + 1))


model.addConstrs(quicksum(t[a, j, k] for k in range(2, n + 1)) == y[a, j] + quicksum(c[j, k] * x[a, j, k]for j in range(i, n+1)) for a in range(1, 3))
#model.addConstrs(quicksum(t[a, j, k]) >= y[a, j] + quicksum(c[j, k] * x[i, j, k]) for i in range(i, n+1) for a in range(1, 3))
#model.addConstrs((x[a, i, j] == 0) or (x[a, i, j] == 1) for a in range(1, 3) for i in range(1, n+1) for j in range(2, n+2))

# Ensure each shopper leaves node one once
model.addConstrs(quicksum(x[1, 1, j]for j in range(2, n + 1)) == 1 for j in range(2, n + 1))
model.addConstrs(quicksum(x[2, 1, j]for j in range(2, n + 1)) == 1 for j in range(2, n + 1))
#model.addConstrs(quicksum(x[1, 1, j]for j in range(2, n + 1)) >= 1 for j in range(2, n + 1))
#model.addConstrs(quicksum(x[2, 1, j]for j in range(2, n + 1)) >= 1 for j in range(2, n + 1))

# Ensure each shopper enters node one once
model.addConstrs(quicksum(x[1, i, 1]for i in range(1, n + 1)) == 1 for i in range(1, n + 1))
model.addConstrs(quicksum(x[2, i, 1]for i in range(1, n + 1)) == 1 for i in range(1, n + 1))
#model.addConstrs(quicksum(x[1, i, 1]for i in range(1, n + 1)) >= 1 for i in range(1, n + 1))
#model.addConstrs(quicksum(x[2, i, 1]for i in range(1, n + 1)) >= 1 for i in range(1, n + 1))

# Shoppers leave and enter other nodes at most once
model.addConstrs(quicksum(x[1, i, j] for j in range(2, n + 1) for i in range(1,n)) <= 1 for i in range (2, n+1))
model.addConstrs(quicksum(x[2, i, j] for j in range(2, n + 1) for i in range(1,n)) <= 1 for i in range (2, n+1))
#model.addConstrs(quicksum(x[1, j, i] for i in range(1, n + 1)) <= 1 for j in range (1, n))
#model.addConstrs(quicksum(x[2, j, i] for i in range(1, n + 1)) <= 1 for j in range (1, n))



# If a shopper enters a node they must leave it
model.addConstrs(quicksum(x[1, i, j] for j in range(1, n)) == quicksum(x[1, j, i] for j in range(1, n)) for i in range (1, n))
#model.addConstrs(quicksum(x[1, i, j] for j in range(1, n + 1)) >= quicksum(x[1, j, i] for j in range(1, n+1)) for i in range (1, n+1))

# We don't use the x[a, i, i] items so that we dont just stay at the same node for no reason
#model.addConstrs(quicksum(x[1, i, i] for i in range (1, n + 1)) < 1)

# if a shopper leaves goes to item j, they must also leave that item

model.optimize()


53
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 16911 rows, 11772 columns and 915422 nonzeros
Model fingerprint: 0xb4c8cecf
Variable types: 5940 continuous, 5832 integer (5832 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [2e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 146.1400000
Presolve removed 16911 rows and 11772 columns
Presolve time: 0.29s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.34 seconds (0.38 work units)
Thread count was 1 (of 10 available processors)

Solution count 2: 201.83 146.14 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.018300000000e+02, best bound 2.018300000000e+02, gap 0.0000%


In [274]:
for var in model.getVars():
    if var.X != 0:
        print(f"{var.VarName}: {var.X}")

x_1_1_22: 1.0
x_1_22_1: 1.0
x_1_40_54: 1.0
x_1_41_54: 1.0
x_1_42_54: 1.0
x_1_43_54: 1.0
x_1_44_54: 1.0
x_1_45_54: 1.0
x_1_46_54: 1.0
x_2_1_22: 1.0
x_2_45_54: 1.0
x_2_46_54: 1.0
x_2_47_54: 1.0
x_2_48_54: 1.0
x_2_49_54: 1.0
x_2_50_54: 1.0
x_2_51_54: 1.0
x_2_52_54: 1.0
x_2_53_1: 1.0


In [276]:
for var in model.getVars():
    if 'y' in var.VarName:
        print(var.VarName)

y_1_54_1
y_1_54_2
y_1_54_3
y_1_54_4
y_1_54_5
y_1_54_6
y_1_54_7
y_1_54_8
y_1_54_9
y_1_54_10
y_1_54_11
y_1_54_12
y_1_54_13
y_1_54_14
y_1_54_15
y_1_54_16
y_1_54_17
y_1_54_18
y_1_54_19
y_1_54_20
y_1_54_21
y_1_54_22
y_1_54_23
y_1_54_24
y_1_54_25
y_1_54_26
y_1_54_27
y_1_54_28
y_1_54_29
y_1_54_30
y_1_54_31
y_1_54_32
y_1_54_33
y_1_54_34
y_1_54_35
y_1_54_36
y_1_54_37
y_1_54_38
y_1_54_39
y_1_54_40
y_1_54_41
y_1_54_42
y_1_54_43
y_1_54_44
y_1_54_45
y_1_54_46
y_1_54_47
y_1_54_48
y_1_54_49
y_1_54_50
y_1_54_51
y_1_54_52
y_1_54_53
y_1_54_54
y_2_54_1
y_2_54_2
y_2_54_3
y_2_54_4
y_2_54_5
y_2_54_6
y_2_54_7
y_2_54_8
y_2_54_9
y_2_54_10
y_2_54_11
y_2_54_12
y_2_54_13
y_2_54_14
y_2_54_15
y_2_54_16
y_2_54_17
y_2_54_18
y_2_54_19
y_2_54_20
y_2_54_21
y_2_54_22
y_2_54_23
y_2_54_24
y_2_54_25
y_2_54_26
y_2_54_27
y_2_54_28
y_2_54_29
y_2_54_30
y_2_54_31
y_2_54_32
y_2_54_33
y_2_54_34
y_2_54_35
y_2_54_36
y_2_54_37
y_2_54_38
y_2_54_39
y_2_54_40
y_2_54_41
y_2_54_42
y_2_54_43
y_2_54_44
y_2_54_45
y_2_54_46
y_2_54_47
y_2_54_4

In [56]:
model.write('TSP.lp')

In [249]:
display(t)

{(1, 1, 1): <gurobi.Var t_1_1_1 (value 0.0)>,
 (1, 1, 2): <gurobi.Var t_1_1_2 (value 0.0)>,
 (1, 1, 3): <gurobi.Var t_1_1_3 (value 0.0)>,
 (1, 1, 4): <gurobi.Var t_1_1_4 (value 0.0)>,
 (1, 1, 5): <gurobi.Var t_1_1_5 (value 0.0)>,
 (1, 1, 6): <gurobi.Var t_1_1_6 (value 0.0)>,
 (1, 1, 7): <gurobi.Var t_1_1_7 (value 0.0)>,
 (1, 1, 8): <gurobi.Var t_1_1_8 (value 0.0)>,
 (1, 1, 9): <gurobi.Var t_1_1_9 (value 0.0)>,
 (1, 1, 10): <gurobi.Var t_1_1_10 (value 0.0)>,
 (1, 1, 11): <gurobi.Var t_1_1_11 (value 0.0)>,
 (1, 1, 12): <gurobi.Var t_1_1_12 (value 0.0)>,
 (1, 1, 13): <gurobi.Var t_1_1_13 (value 0.0)>,
 (1, 1, 14): <gurobi.Var t_1_1_14 (value 0.0)>,
 (1, 1, 15): <gurobi.Var t_1_1_15 (value 0.0)>,
 (1, 1, 16): <gurobi.Var t_1_1_16 (value 0.0)>,
 (1, 1, 17): <gurobi.Var t_1_1_17 (value 0.0)>,
 (1, 1, 18): <gurobi.Var t_1_1_18 (value 0.0)>,
 (1, 1, 19): <gurobi.Var t_1_1_19 (value 0.0)>,
 (1, 1, 20): <gurobi.Var t_1_1_20 (value 0.0)>,
 (1, 1, 21): <gurobi.Var t_1_1_21 (value 0.0)>,
 (1, 1, 22

In [207]:
#part e formulation

model.setObjective(quicksum(prices[i] * x[a, i, j] for i in range(1, n+1) for j in range(2, n+2) for a in range(1, 3)), GRB.MAXIMIZE)

#constraint 1 - shopper 1 has no more than one minute
model.addConstr(0.1 * y[n+1] + quicksum(2 * x[i, j] for i in range(1, n) for j in range(2, n+2)) <= 60)

#constraint 2 - basket limit
model.addConstr(quicksum(x[i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 10)
                   
#constraint 3 - no same item twice 
model.addConstrs(quicksum(x[i, j] for i in range(1, n+1)) <= 1 for j in range(2, n+2))

#TSP constraints 
model.addConstrs(t[i, j] <= M * x[i, j] for i in range(1, n+1) for j in range(2, n+2))
model.addConstr(y[1] == 0)
model.addConstrs(y[j] == quicksum(t[i, j] for i in range(1, n+1)) for j in range(1, n+2))
model.addConstrs((x[i, j] == 0) or (x[i, j] == 1) for i in range(1, n+1) for j in range(2, n+2))

#adding the dij constraint
model.addConstrs(d[i] == 0 for i in range(1, n+1) if x[i, 2] == 1)
model.addConstrs(d[i] >= 1 for i in range(1, n+1) if x[i, 2] == 0)

#the rest hehe

model.setObjective(quicksum(prices[i] * x[a, i, j] for i in range(1, n+1) for j in range(2, n+2) for a in range(1, 3)), GRB.MAXIMIZE)


#constraint 1 - shopper 1 has no more than one minute
model.addConstr(0.1 * y[n+1] + quicksum(2 * x[i, j] for i in range(1, n) for j in range(2, n+2)) <= 60)

#constraint 2 - basket limit
model.addConstr(quicksum(x[i, j] for i in range(1, n+1) for j in range(2, n+2)) <= 10)
                   
#constraint 3 - no same item twice 
model.addConstrs(quicksum(x[i, j] for i in range(1, n+1)) <= 1 for j in range(2, n+2))

#TSP constraints 
model.addConstrs(t[i, j] <= M * x[i, j] for i in range(1, n+1) for j in range(2, n+2))
model.addConstr(y[1] == 0)
model.addConstrs(y[j] == quicksum(t[i, j] for i in range(1, n+1)) for j in range(1, n+2))
model.addConstrs((x[i, j] == 0) or (x[i, j] == 1) for i in range(1, n+1) for j in range(2, n+2))




KeyError: 57

In [265]:
help(model)


  Gurobi model object.  Commonly used methods on this object are:
    getConstrs(): Get a list of constraints in the model
    getJSONSolution(): Get a JSON-string representation of the current solution(s) to the model
    getParamInfo(paramname): Get information on a model parameter.
    getVars(): Get a list of variables in the model
    optimize(): Optimize the model.
    printAttr(attrname, filter): Print attribute values.
    printQuality(): Print solution quality statistics.
    printStats(): Print model statistics.
    read(filename): Read model data (MIP start, basis, etc.) from a file
    reset(): Discard any resident solution information.
    resetParams(): Reset all parameters to their default values.
    setParam(paramname, newval): Set a model parameter to a new value.
    write(filename): Write model data to a file.

  Models have a number of attributes that can be queried or modified.
  For example, "print model.objval" prints the objective value of
  the current soluti